In [1]:
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt
import scipy.io as io

In [22]:
def res(X,mu, sigma, Pi):
#     print(X.shape)
#     print(mu.shape)
#     print(sigma.shape)
#     print(Pi)
    d = Pi*sp.multivariate_normal.pdf(X, mu, sigma)
    return d

In [2]:
full = io.loadmat('shuttle.mat')

In [231]:
def TTsplit(matrix, train = .7):
    #     This function's purpose is to split a given matrix
    #     into three parts: train, test, and validate. The 
    #     train parameter is the percentage of the data that
    #     goes into training 70% by default. The val parameter
    #     determines how much goes into the validation set
    #     15% by default. The rest of the data goes into
    #     the test set.
    
#     The following is needed to perform the operations
#     in the calculations in here.
    import numpy as np
    # The following does some common sense things like
    # telling you that train and val need to add up to less
    # than 1.

    if(train > .99):
        print("Splitting failed. Please make sure that train is less than .99 so that test and val have enough data.")
        return 0
    
#     These two things calculate the subset lengths that 
#     are needed for the subsetting.
    splitnum = int(len(matrix[:,0])*train)
    np.random.shuffle(matrix)

#     The next three separate the data into the three sets
#     using the splitnums from above.
    Train = matrix[0:splitnum,]
    Test = matrix[splitnum:,]
    return Train, Test

In [235]:
fullX = np.array(full['X'], dtype = float)
fullY = full['y']

In [236]:
train, test = TTsplit(np.column_stack((fullX,fullY)))

In [238]:
Xtrain = train[np.where(train[:,-1] == 0)[0],:-1]


In [100]:
len(fullX)

49097

In [478]:
k = 14
M = np.empty((k,Xtrain.shape[1]))
sigma = np.empty((Xtrain.shape[1],Xtrain.shape[1],k))
M = Xtrain[np.random.choice(Xtrain.shape[0],k)].astype(np.float64)
for i in range(k):
    r = np.random.randn(9,9)*500
    sigma[:,:,i] = r.T@r
Pi = np.ones(k)/k
for i in range(40):
    print(i)
    D = np.empty((len(Xtrain),k))
    for i in range(0,k):
        D[:,i] = np.array(res(Xtrain, M[i,], sigma[:,:,i], Pi[i]), dtype = float)
    D = D/np.sum(D, axis = 1).reshape(-1,1)
    Pi = np.mean(D,axis = 0)
    for i in range(0,k):
        M[i,] = np.array(np.sum(D[:,i].reshape(-1,1)*Xtrain, axis = 0)/np.sum(D[:,i]).reshape(-1,1), dtype = float)
        sigma[:,:,i] = (D[:,i].reshape(-1,1)*(Xtrain - M[i,])).T@(Xtrain - M[i,])/np.sum(D[:,i]).reshape(-1,1) + .00001*np.eye(fullX.shape[1])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [479]:
newD = np.empty((len(train[:,:-1]),k))
for i in range(0,k):
    newD[:,i] = np.array(res(train[:,:-1], M[i,], sigma[:,:,i], Pi[i]), dtype = float)

# The Following Calculates The Number of Misclassified Points in The Training Set

In [485]:
len(train[:,-1])-np.sum((np.sum(newD, axis = 1)<1e-17).reshape(-1,1) == train[:,-1].reshape(-1,1))

30

In [486]:
newestD = np.empty((len(test[:,:-1]),k))
for i in range(0,k):
    newestD[:,i] = np.array(res(test[:,:-1], M[i,], sigma[:,:,i], Pi[i]), dtype = float)

In [487]:
len(test[:,-1])-np.sum((np.sum(newestD, axis = 1)<1e-17).reshape(-1,1) == test[:,-1].reshape(-1,1))

36